# Importing Libraries

In [ ]:
from keras.models import load_model
import numpy as np
import pandas as pd
import cv2
import csv
import os 
from os import walk

# Defining functions

In [84]:
def get_img_paths(family_filepath):
    
    fish = []

    for (dirpath, dirnames, filenames) in walk(family_filepath):
        for i in range(len(filenames)):
            img = dirpath + filenames[i]
            if "Store" not in img and ".csv" not in img:  
                fish.append(img)
        break
    
    return fish, filenames

def getFish(filepath, family):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    if "noeyespot" in filepath:
        classification = "noeyespot"
    else:
        classification = "eyespot"  
    
    _, _, species = filepath.partition(family+'/')
    
    #res = model.predict(img)
    return(img, classification, species)

def predict(model, get_fish_output):
    
    img, correct_class, fam = get_fish_output
    
    prediction_weights = model.predict(img)
    prediction = model.predict_classes(img)
    if (prediction[0] == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"

    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, prediction_weights, fam)

# Loading model

In [11]:
mod = load_model("10x-classifier.h5")

# Loading filepaths

In [85]:
family = 'Labridae2'

family_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/randall/Labridae2/"

fish_list_img, fish_img_names = get_img_paths(family_path)

_,_,weights,_ = predict(mod,getFish(fish_list_img[1],'Labridae'))
weights = np.array2string(weights)
weights

weights = (weights.split('[['))[1].split(']]')[0]

In [75]:
os.chdir(family_path)

# Prediction over family and write to CSV

In [87]:
with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Prediction Weights', 'Species'])
    
    try:
        os.mkdir('eyespot')
        os.mkdir('noeyespot')
    except OSError:
        1
      #  print "unsuccessful"

    for i in range(len(fish_list_img)):
        classification,_,weights,family = predict(mod, getFish(fish_list_img[i], 'Labridae2'))
        species = family.split('_')[1]
        filewriter.writerow([classification, weights, species])
        
        #weights = np.array2string(weights)
        #weights = (weights.split('[['))[1].split(']]')[0]
        
        
        if (classification == 'eyespot'):
            try:
                os.rename(fish_img_names[i], 'eyespot/' + np.array2string(weights) + "_" + fish_img_names[i])
            except OSError:
                1
        else:
            try:
                os.rename(fish_img_names[i], 'noeyespot/' + np.array2string(weights) + "_" + fish_img_names[i])
            except OSError:
                1

error: /opt/concourse/worker/volumes/live/3b96f7c7-93a4-48c6-665d-2f7ff1dac914/volume/opencv_1512680443756/work/modules/imgproc/src/resize.cpp:3289: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
